In [1]:
import os 
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES']='8'

In [2]:
# GPU 확인
import torch
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
print(f"사용 가능한 GPU 개수: {torch.cuda.device_count()}")
if torch.cuda.is_available():
    print(f"현재 GPU: {torch.cuda.get_device_name(0)}")

CUDA 사용 가능: True
사용 가능한 GPU 개수: 1
현재 GPU: Tesla V100-PCIE-32GB


In [10]:
import cv2
import os

# opencv version확인
print(cv2.__version__)

# 비디오 가져오기
filepath = "./datasets/test.mp4"
video = cv2.VideoCapture(filepath)

if not video.isOpened():
    print("Could not Open :", filepath)
    exit(0)

#불러온 비디오 파일의 정보 출력
length = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = video.get(cv2.CAP_PROP_FPS)

print("length :", length)
print("width :", width)
print("height :", height)
print("fps :", fps)

#프레임을 저장할 디렉토리를 생성
try:
    if not os.path.exists(filepath[:-4]):
        os.makedirs(filepath[:-4])
except OSError:
    print ('Error: Creating directory. ' +  filepath[:-4])

count = 0

while video.isOpened():
    ret, image = video.read()
    if not ret:  # 비디오가 끝나면 종료
        break

    # 현재 프레임 번호를 가져와 fps로 나누어 떨어지는 프레임마다 저장
    frame_id = int(video.get(cv2.CAP_PROP_POS_FRAMES))
    if frame_id % (video.get(cv2.CAP_PROP_FPS) // fps) == 0:
        cv2.imwrite(filepath[:-4] + "/frame%d.jpg" % count, image)
        print('Saved frame number:', str(frame_id))
        count += 1
# 비디오 메모리 해제
video.release()


4.10.0
length : 60
width : 2304
height : 1280
fps : 15.0
Saved frame number: 1
Saved frame number: 2
Saved frame number: 3
Saved frame number: 4
Saved frame number: 5
Saved frame number: 6
Saved frame number: 7
Saved frame number: 8
Saved frame number: 9
Saved frame number: 10
Saved frame number: 11
Saved frame number: 12
Saved frame number: 13
Saved frame number: 14
Saved frame number: 15
Saved frame number: 16
Saved frame number: 17
Saved frame number: 18
Saved frame number: 19
Saved frame number: 20
Saved frame number: 21
Saved frame number: 22
Saved frame number: 23
Saved frame number: 24
Saved frame number: 25
Saved frame number: 26
Saved frame number: 27
Saved frame number: 28
Saved frame number: 29
Saved frame number: 30
Saved frame number: 31
Saved frame number: 32
Saved frame number: 33
Saved frame number: 34
Saved frame number: 35
Saved frame number: 36
Saved frame number: 37
Saved frame number: 38
Saved frame number: 39
Saved frame number: 40
Saved frame number: 41
Saved fra

In [11]:
import os
import cv2
import subprocess

# 파일 경로 설정
video_path = "datasets/test.mp4"  # 입력 비디오 파일 경로
output_dir = "datasets"  # 출력 디렉토리 경로
resolution = "4k (3840 x 2160)"  # 해상도 선택
model = "RealESRGAN_x4plus_anime_6B"  # 업스케일링 모델 선택 ["RealESRGAN_x4plus" , "RealESRGAN_x4plus_anime_6B", "realesr-animevideov3"]

# 비디오 파일 존재 여부 확인
assert os.path.exists(video_path), "비디오 파일이 존재하지 않습니다."

# 비디오 해상도 정보 가져오기
video_capture = cv2.VideoCapture(video_path)
video_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
video_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

# 최종 해상도 설정
final_width, final_height = None, None
aspect_ratio = float(video_width / video_height)

# 해상도 매칭
if resolution == "FHD (1920 x 1080)":
    final_width, final_height = 1920, 1080
elif resolution == "2k (2560 x 1440)":
    final_width, final_height = 2560, 1440
elif resolution == "4k (3840 x 2160)":
    final_width, final_height = 3840, 2160
elif resolution == "2 x original":
    final_width, final_height = 2 * video_width, 2 * video_height
elif resolution == "3 x original":
    final_width, final_height = 3 * video_width, 3 * video_height
elif resolution == "4 x original":
    final_width, final_height = 4 * video_width, 4 * video_height

# 비율 조정
if aspect_ratio == 1.0 and "original" not in resolution:
    final_height = final_width
elif aspect_ratio < 1.0 and "original" not in resolution:
    final_width, final_height = final_height, final_width

# 스케일 팩터 계산
scale_factor = max(final_width / video_width, final_height / video_height)
is_even = int(video_width * scale_factor) % 2 == 0 and int(video_height * scale_factor) % 2 == 0

# 스케일 팩터가 짝수가 될 때까지 조정
while not is_even:
    scale_factor += 0.01
    is_even = int(video_width * scale_factor) % 2 == 0 and int(video_height * scale_factor) % 2 == 0

print(f"업스케일링 시작: {video_width}x{video_height} -> {final_width}x{final_height}, 스케일 팩터={scale_factor}")

# Real-ESRGAN을 통한 업스케일링 명령어 실행
command = f"python inference_realesrgan_video.py -n {model} -i '{video_path}' -o '{output_dir}' --outscale {scale_factor}"
subprocess.run(command, shell=True)

# 비디오 이름 및 업스케일링 경로 설정
video_name_with_ext = os.path.basename(video_path)
video_name = video_name_with_ext.replace(".mp4", "")
upscaled_video_path = f"{output_dir}{video_name}_out.mp4"
final_video_name = f"{video_name}_upscaled_{final_width}_{final_height}.mp4"
final_video_path = os.path.join(output_dir, final_video_name)

# 지정된 해상도로 크롭하여 맞추기
if "original" not in resolution:
    print("지정된 해상도로 크롭 중...")
    crop_command = f"ffmpeg -loglevel error -y -i '{upscaled_video_path}' -c:v h264_nvenc -filter:v 'crop={final_width}:{final_height}:(in_w-{final_width})/2:(in_h-{final_height})/2' -c:v libx264 -pix_fmt yuv420p '{final_video_path}'"
    subprocess.run(crop_command, shell=True)

print(f"업스케일링된 비디오 저장 경로: {final_video_path}")

# 중간 업스케일링 파일 삭제
if os.path.exists(upscaled_video_path):
    os.remove(upscaled_video_path)


업스케일링 시작: 2304x1280 -> 3840x2160, 스케일 팩터=1.6875
지정된 해상도로 크롭 중...
업스케일링된 비디오 저장 경로: datasets/test_upscaled_3840_2160.mp4


Traceback (most recent call last):
  File "/home/j-k11a604/inference_realesrgan_video.py", line 2, in <module>
    import cv2
ModuleNotFoundError: No module named 'cv2'
datasetstest_out.mp4: No such file or directory


In [16]:
import torch
assert torch.cuda.is_available(), "GPU not detected.. Please change runtime to GPU"

from PIL import Image
import cv2, os, subprocess
from tqdm import tqdm

In [23]:
!pip install opencv-python
!pip install basicsr
!pip install facexlib
!pip install gfpgan
!pip install numpy
!pip install Pillow
!pip install torch torchvision
!pip install ffmpeg-python

Defaulting to user installation because normal site-packages is not writeable
  Using cached opencv_python-4.10.0.84-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
Using cached opencv_python-4.10.0.84-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (62.5 MB)

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
  Using cached basicsr-1.4.2.tar.gz (172 kB)
  Preparing metadata (setup.py) ... done
  Using cached addict-2.4.0-py3-none-any.whl.metadata (1.0 kB)
  Using cached future-1.0.0-py3-none-any.whl.metadata (4.0 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.0 MB/s eta 0:00:00
  Using cached tb_nightly-2.19.0a20241029-py3-none-any.whl.metadata (1.6 kB)
  Using cached yapf-0.40.2-py3-none-any.whl.metadata (45 kB)
  Using cached imageio-2.36.0-py3-none-any.whl.metadata (5.2 kB)
  Using cached lazy_l

In [4]:
import os
import cv2
import subprocess

# Define video path and output directory
video_path = "./datasets/test.mp4"  # Change to your dataset path
output_dir = "./datasets"  # Output directory for upscaled video
resolution = "4k (3840 x 2160)"  # Change as needed
model = "RealESRGAN_x4plus_anime_6B"  # Change model as needed

# Ensure the video file exists
assert os.path.exists(video_path), "Video file does not exist"

# Get video properties
video_capture = cv2.VideoCapture(video_path)
video_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
video_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

final_width = None
final_height = None
aspect_ratio = float(video_width / video_height)

# Set output resolutions
if resolution == "FHD (1920 x 1080)":
    final_width = 1920
    final_height = 1080
elif resolution == "2k (2560 x 1440)":
    final_width = 2560
    final_height = 1440
elif resolution == "4k (3840 x 2160)":
    final_width = 3840
    final_height = 2160
elif resolution == "2 x original":
    final_width = 2 * video_width
    final_height = 2 * video_height
elif resolution == "3 x original":
    final_width = 3 * video_width
    final_height = 3 * video_height
elif resolution == "4 x original":
    final_width = 4 * video_width
    final_height = 4 * video_height

# Adjust final height for square aspect ratios
if aspect_ratio == 1.0 and "original" not in resolution:
    final_height = final_width

if aspect_ratio < 1.0 and "original" not in resolution:
    temp = final_width
    final_width = final_height
    final_height = temp

# Calculate scale factor
scale_factor = max(final_width / video_width, final_height / video_height)
isEven = int(video_width * scale_factor) % 2 == 0 and int(video_height * scale_factor) % 2 == 0

# Ensure scale factor is even
while not isEven:
    scale_factor += 0.01
    isEven = int(video_width * scale_factor) % 2 == 0 and int(video_height * scale_factor) % 2 == 0

print(f"Upscaling from {video_width}x{video_height} to {final_width}x{final_height}, scale_factor={scale_factor}")

# Run inference for video upscaling
subprocess.run(
    f"python Real-ESRGAN/inference_realesrgan_video.py -n {model} -i '{video_path}' -o '{output_dir}' --outscale {scale_factor} --tile 128",
    shell=True
)

video_name_with_ext = os.path.basename(video_path)
video_name = video_name_with_ext.replace(".mp4", "")
upscaled_video_path = os.path.join(output_dir, f"{video_name}_out.mp4")
final_video_name = f"{video_name}_upscaled_{final_width}_{final_height}.mp4"
final_video_path = os.path.join(output_dir, final_video_name)

# Crop to fit the final resolution
if "original" not in resolution:
    print("Cropping to fit...")
    command = f"ffmpeg -loglevel error -y -i '{upscaled_video_path}' -c:v h264_nvenc -filter:v 'crop={final_width}:{final_height}:(in_w-{final_width})/2:(in_h-{final_height})/2' -c:v libx264 -pix_fmt yuv420p '{final_video_path}'"
    subprocess.run(command, shell=True)

print(f"Upscaled video saved to: {final_video_path}")

# Remove the upscaled video to clean up
# os.remove(upscaled_video_path)


Upscaling from 2304x1280 to 3840x2160, scale_factor=1.6875


inference:   5%|▌         | 3/60 [00:18<05:41,  6.00s/frame]

	Tile 1/180
	Tile 2/180
	Tile 3/180
	Tile 4/180
	Tile 5/180
	Tile 6/180
	Tile 7/180
	Tile 8/180
	Tile 9/180
	Tile 10/180
	Tile 11/180
	Tile 12/180
	Tile 13/180
	Tile 14/180
	Tile 15/180
	Tile 16/180
	Tile 17/180
	Tile 18/180
	Tile 19/180
	Tile 20/180
	Tile 21/180
	Tile 22/180
	Tile 23/180
	Tile 24/180
	Tile 25/180
	Tile 26/180
	Tile 27/180
	Tile 28/180
	Tile 29/180
	Tile 30/180
	Tile 31/180
	Tile 32/180
	Tile 33/180
	Tile 34/180
	Tile 35/180
	Tile 36/180
	Tile 37/180
	Tile 38/180
	Tile 39/180
	Tile 40/180
	Tile 41/180
	Tile 42/180
	Tile 43/180
	Tile 44/180
	Tile 45/180
	Tile 46/180
	Tile 47/180
	Tile 48/180
	Tile 49/180
	Tile 50/180
	Tile 51/180
	Tile 52/180
	Tile 53/180
	Tile 54/180
	Tile 55/180
	Tile 56/180
	Tile 57/180
	Tile 58/180
	Tile 59/180
	Tile 60/180
	Tile 61/180
	Tile 62/180
	Tile 63/180
	Tile 64/180
	Tile 65/180
	Tile 66/180
	Tile 67/180
	Tile 68/180
	Tile 69/180
	Tile 70/180
	Tile 71/180
	Tile 72/180
	Tile 73/180
	Tile 74/180
	Tile 75/180
	Tile 76/180
	Tile 77/180
	Tile 78

inference:  10%|█         | 6/60 [00:34<04:57,  5.51s/frame]

	Tile 75/180
	Tile 76/180
	Tile 77/180
	Tile 78/180
	Tile 79/180
	Tile 80/180
	Tile 81/180
	Tile 82/180
	Tile 83/180
	Tile 84/180
	Tile 85/180
	Tile 86/180
	Tile 87/180
	Tile 88/180
	Tile 89/180
	Tile 90/180
	Tile 91/180
	Tile 92/180
	Tile 93/180
	Tile 94/180
	Tile 95/180
	Tile 96/180
	Tile 97/180
	Tile 98/180
	Tile 99/180
	Tile 100/180
	Tile 101/180
	Tile 102/180
	Tile 103/180
	Tile 104/180
	Tile 105/180
	Tile 106/180
	Tile 107/180
	Tile 108/180
	Tile 109/180
	Tile 110/180
	Tile 111/180
	Tile 112/180
	Tile 113/180
	Tile 114/180
	Tile 115/180
	Tile 116/180
	Tile 117/180
	Tile 118/180
	Tile 119/180
	Tile 120/180
	Tile 121/180
	Tile 122/180
	Tile 123/180
	Tile 124/180
	Tile 125/180
	Tile 126/180
	Tile 127/180
	Tile 128/180
	Tile 129/180
	Tile 130/180
	Tile 131/180
	Tile 132/180
	Tile 133/180
	Tile 134/180
	Tile 135/180
	Tile 136/180
	Tile 137/180
	Tile 138/180
	Tile 139/180
	Tile 140/180
	Tile 141/180
	Tile 142/180
	Tile 143/180
	Tile 144/180
	Tile 145/180
	Tile 146/180
	Tile 147/180
	Ti

...

	Tile 165/180
	Tile 166/180
	Tile 167/180
	Tile 168/180
	Tile 169/180
	Tile 170/180
	Tile 171/180
	Tile 172/180
	Tile 173/180
	Tile 174/180
	Tile 175/180
	Tile 176/180
	Tile 177/180
	Tile 178/180
	Tile 179/180
	Tile 180/180
	Tile 1/180
	Tile 2/180
	Tile 3/180
	Tile 4/180
	Tile 5/180
	Tile 6/180
	Tile 7/180
	Tile 8/180
	Tile 9/180
	Tile 10/180
	Tile 11/180
	Tile 12/180
	Tile 13/180
	Tile 14/180
	Tile 15/180
	Tile 16/180
	Tile 17/180
	Tile 18/180
	Tile 19/180
	Tile 20/180
	Tile 21/180
	Tile 22/180
	Tile 23/180
	Tile 24/180
	Tile 25/180
	Tile 26/180
	Tile 27/180
	Tile 28/180
	Tile 29/180
	Tile 30/180
	Tile 31/180
	Tile 32/180
	Tile 33/180
	Tile 34/180
	Tile 35/180
	Tile 36/180
	Tile 37/180
	Tile 38/180
	Tile 39/180
	Tile 40/180
	Tile 41/180
	Tile 42/180
	Tile 43/180
	Tile 44/180
	Tile 45/180
	Tile 46/180
	Tile 47/180
	Tile 48/180
	Tile 49/180
	Tile 50/180
	Tile 51/180
	Tile 52/180
	Tile 53/180
	Tile 54/180
	Tile 55/180
	Tile 56/180
	Tile 57/180
	Tile 58/180
	Tile 59/180
	Tile 60/180
	Tile

inference:  95%|█████████▌| 57/60 [05:06<00:15,  5.32s/frame]

	Tile 58/180
	Tile 59/180
	Tile 60/180
	Tile 61/180
	Tile 62/180
	Tile 63/180
	Tile 64/180
	Tile 65/180
	Tile 66/180
	Tile 67/180
	Tile 68/180
	Tile 69/180
	Tile 70/180
	Tile 71/180
	Tile 72/180
	Tile 73/180
	Tile 74/180
	Tile 75/180
	Tile 76/180
	Tile 77/180
	Tile 78/180
	Tile 79/180
	Tile 80/180
	Tile 81/180
	Tile 82/180
	Tile 83/180
	Tile 84/180
	Tile 85/180
	Tile 86/180
	Tile 87/180
	Tile 88/180
	Tile 89/180
	Tile 90/180
	Tile 91/180
	Tile 92/180
	Tile 93/180
	Tile 94/180
	Tile 95/180
	Tile 96/180
	Tile 97/180
	Tile 98/180
	Tile 99/180
	Tile 100/180
	Tile 101/180
	Tile 102/180
	Tile 103/180
	Tile 104/180
	Tile 105/180
	Tile 106/180
	Tile 107/180
	Tile 108/180
	Tile 109/180
	Tile 110/180
	Tile 111/180
	Tile 112/180
	Tile 113/180
	Tile 114/180
	Tile 115/180
	Tile 116/180
	Tile 117/180
	Tile 118/180
	Tile 119/180
	Tile 120/180
	Tile 121/180
	Tile 122/180
	Tile 123/180
	Tile 124/180
	Tile 125/180
	Tile 126/180
	Tile 127/180
	Tile 128/180
	Tile 129/180
	Tile 130/180
	Tile 131/180
	Tile 

inference: 100%|██████████| 60/60 [05:24<00:00,  5.41s/frame]


	Tile 129/180
	Tile 130/180
	Tile 131/180
	Tile 132/180
	Tile 133/180
	Tile 134/180
	Tile 135/180
	Tile 136/180
	Tile 137/180
	Tile 138/180
	Tile 139/180
	Tile 140/180
	Tile 141/180
	Tile 142/180
	Tile 143/180
	Tile 144/180
	Tile 145/180
	Tile 146/180
	Tile 147/180
	Tile 148/180
	Tile 149/180
	Tile 150/180
	Tile 151/180
	Tile 152/180
	Tile 153/180
	Tile 154/180
	Tile 155/180
	Tile 156/180
	Tile 157/180
	Tile 158/180
	Tile 159/180
	Tile 160/180
	Tile 161/180
	Tile 162/180
	Tile 163/180
	Tile 164/180
	Tile 165/180
	Tile 166/180
	Tile 167/180
	Tile 168/180
	Tile 169/180
	Tile 170/180
	Tile 171/180
	Tile 172/180
	Tile 173/180
	Tile 174/180
	Tile 175/180
	Tile 176/180
	Tile 177/180
	Tile 178/180
	Tile 179/180
	Tile 180/180
	Tile 1/180
	Tile 2/180
	Tile 3/180
	Tile 4/180
	Tile 5/180
	Tile 6/180
	Tile 7/180
	Tile 8/180
	Tile 9/180
	Tile 10/180
	Tile 11/180
	Tile 12/180
	Tile 13/180
	Tile 14/180
	Tile 15/180
	Tile 16/180
	Tile 17/180
	Tile 18/180
	Tile 19/180
	Tile 20/180
	Tile 21/180
	Tile 22

In [ ]:
!pwd